## 分類用データセット作成Notebook
MovieLensには正例（実際にCVしたレコード）しか含まれていないためネガティブサンプルを作成する。   
またTrain/Eval/Testで分割するときにtimestampでソートしてから適当な数で分割することとしている（今回はTrain:Eval:Test=7:2:1ぐらいの比率で分割）

In [8]:
import random
from time import time
import numpy as np
import pandas as pd

In [9]:
data_set_file_path = '/Users/fumiyo_ito/Documents/git/Wide-Deep/data/MovieLens20M/'

元データの読み込み

In [11]:
rec_dataset = pd.read_csv(data_set_file_path + 'rating.csv', names=['user','item','rating','timestamp'], skiprows=1)
#rec_dataset = pd.read_csv('../data/u.data', sep='\t',
                         #header=None,
                         #names=['user','item','rating','timestamp'])
rec_dataset.head()

,user,item,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [12]:
print('Dataset size: {}'.format(len(rec_dataset)))

Dataset size: 20000263


以下のデータ数でサンプルを作成する（以下はそれぞれの数字／実際はこれの倍の大きさ）   

- MovieLens20M
    - Train: 14,000,000    
    - Eval: 4,000,000    
    - Test: 2,000,263
- MovieLens100k
    - Train: 70,000
    - Eval: 20,000
    - TestL 10,000

### ユーザ・アイテムの組み合わせを適当に作成する関数   
ユーザ・アイテムの辞書を作成しておき、適当に作成された組み合わせがその中に存在していなければリストに加えるというもの。   
ここで作成したリストはあとでDataFrameに変換する

In [13]:
def create_random_combinations(df):
    pos_set = set(df.iloc[:,:2].itertuples(index=False, name=None))
    user_list = df.user.unique()
    item_list = df.item.unique()
    neg_user = []
    neg_item = []
    
    while len(neg_user) < len(df):
        new_user = random.choice(user_list)
        new_item = random.choice(item_list)
        new_sample = (new_user, new_item)
        if new_sample not in pos_set:
            neg_user.append(new_user)
            neg_item.append(new_item)
    return neg_user, neg_item

In [14]:
t1 = time()
neg_user, neg_item = create_random_combinations(rec_dataset)
t2 = time()
print('{:.4f} seconds elapsed'.format(t2-t1))

86.7178 seconds elapsed


In [15]:
neg_df = pd.DataFrame(np.concatenate([np.array(neg_user).reshape(-1,1),
                         np.array(neg_item).reshape(-1,1),
                         np.zeros(len(neg_user)).reshape(-1,1)
                        ], axis=1),
                      columns=['user','item','rating'])

neg_df.head()

,user,item,rating
0,58725.0,27751.0,0.0
1,8539.0,6626.0,0.0
2,73551.0,55245.0,0.0
3,101245.0,90568.0,0.0
4,80650.0,31545.0,0.0


In [16]:
# データ型を整える
neg_df.iloc[:,0] = neg_df.iloc[:,0].map(int).map(str)
neg_df.iloc[:,1] = neg_df.iloc[:,1].map(int).map(str)
neg_df.iloc[:,2] = neg_df.iloc[:,2].map(int)
neg_df.head()


,user,item,rating
0,58725,27751,0
1,8539,6626,0
2,73551,55245,0
3,101245,90568,0
4,80650,31545,0


In [17]:
# 元のデータをpositiveデータとするためratingを1.0に変更しdf化

pos_df = rec_dataset.sort_values('timestamp').iloc[:,:3]
del rec_dataset

# データ型を整える
pos_df.iloc[:,0] = pos_df.iloc[:,0].map(int).map(str)
pos_df.iloc[:,1] = pos_df.iloc[:,1].map(int).map(str)
pos_df.iloc[:,2] = pos_df.iloc[:,2].map(int)
pos_df.loc[:,'rating'] = 1

pos_df.head()


,user,item,rating
4182421,28507,1176,1
18950979,131160,1079,1
18950936,131160,47,1
18950930,131160,21,1
12341178,85252,45,1


In [18]:
# Train用のデータセット作成
train_file_name = data_set_file_path + 'train100k.csv'
pos_df_train = pos_df.iloc[:14000000,:]
neg_df_train = neg_df.iloc[:14000000,:]

# 特に意味はないが固まっているのが気になるためランダムソートしている
dataset_train = pd.concat([pos_df_train, neg_df_train], axis=0).sample(frac=1).reset_index(drop=True)
dataset_train.to_csv(train_file_name, index=None)


In [19]:
# Eval用のデータセット作成
eval_file_name = data_set_file_path + 'eval100k.csv'
pos_df_eval = pos_df.iloc[14000000:18000000,:]
neg_df_eval = neg_df.iloc[14000000:18000000,:]

dataset_eval = pd.concat([pos_df_eval, neg_df_eval], axis=0).sample(frac=1).reset_index(drop=True)
dataset_eval.to_csv(eval_file_name, index=None)


In [20]:
# Test用のデータセット作成
test_file_name = data_set_file_path + 'test100k.csv'
pos_df_test = pos_df.iloc[18000000:,:]
neg_df_test = neg_df.iloc[18000000:,:]

dataset_test = pd.concat([pos_df_test, neg_df_test], axis=0).sample(frac=1).reset_index(drop=True)
dataset_test.to_csv(test_file_name, index=None)


In [21]:
# 中身の確認
filename = test_file_name
check_df = pd.read_csv(filename)
print('Dataset size: {}'.format(len(check_df)))
print(check_df.dtypes)
check_df.head()

Dataset size: 4000526
user      int64
item      int64
rating    int64
dtype: object


,user,item,rating
0,10765,2115,1
1,129611,57669,1
2,10989,2218,0
3,19571,73480,0
4,112502,78064,0


In [22]:
# 目的変数がstrでなければならない場合があるため一旦別ファイルを作成
# 入っているレコードは同じ
train_file_name = data_set_file_path + 'train100k.csv'
train_file_name_str = data_set_file_path + 'train100k.csv'
tmp_df = pd.read_csv(train_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(train_file_name_str, index=None)

In [23]:
# Eval
eval_file_name = data_set_file_path + 'eval100k.csv'
eval_file_name_str = data_set_file_path + '/eval100k.csv'
tmp_df = pd.read_csv(eval_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(eval_file_name_str, index=None)

In [24]:
# Test
test_file_name = data_set_file_path + '/test100k.csv'
test_file_name_str = data_set_file_path + '/test100k.csv'
tmp_df = pd.read_csv(test_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(test_file_name_str, index=None)